In [10]:
import logging
import time
from pathlib import Path
from typing import Optional, Dict, List, Any
from dataclasses import dataclass
from datetime import datetime
import tiktoken
import requests
import json
import os
from tqdm import tqdm
import ollama
from pydantic import BaseModel
from dotenv import load_dotenv

from GatenlpUtils import loadCorpus


# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('ie_processing.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

@dataclass
class ProcessingConfig:
    """Configuration for the IE processing pipeline"""
    max_documents: int = 10
    max_retries: int = 3
    retry_delay: float = 1.0
    temperature: float = 0.0
    output_dir: str = "output"
    backup_dir: str = "backup"
    via_web: bool = False
    batch_size: int = 1
    reserve_tokens: int = 1000
    
    def __post_init__(self):
        # Create output directories
        Path(self.output_dir).mkdir(exist_ok=True)
        Path(self.backup_dir).mkdir(exist_ok=True)

class TokenCounter:
    """Utility class for counting tokens accurately"""
    
    def __init__(self):
        try:
            self.encoding = tiktoken.encoding_for_model("gpt-4")
        except:
            self.encoding = tiktoken.get_encoding("cl100k_base")
    
    def count_tokens(self, text: str) -> int:
        """Count tokens in text"""
        try:
            return len(self.encoding.encode(text))
        except:
            return len(text) // 4  # Fallback estimation
    
    def check_context_fit(self, text: str, max_context: int, reserve_tokens: int = 1000) -> bool:
        """Check if text fits within context length"""
        tokens = self.count_tokens(text)
        return tokens <= (max_context - reserve_tokens)

class ModelManager:
    """Manages model configurations and context lengths"""
    
    MODEL_CONTEXTS = {
        "gemma3:1b": 8192,
        "gemma3:12b": 8192,
        "mistral:latest": 32768,
        "llama3.3:latest": 128000,
        "deepseek-r1:8b": 128000,
        "chevalblanc/claude-3-haiku:latest": 200000,
        "incept5/llama3.1-claude:latest": 128000,
    }
    
    def __init__(self):
        self.token_counter = TokenCounter()
    
    def get_context_length(self, model: str) -> int:
        """Get context length for a model"""
        return self.MODEL_CONTEXTS.get(model, 8192)  # Default to 8192
    
    def can_process_text(self, model: str, text: str, reserve_tokens: int = 1000) -> bool:
        """Check if model can process the given text"""
        context_length = self.get_context_length(model)
        return self.token_counter.check_context_fit(text, context_length, reserve_tokens)

# Initialize components
config = ProcessingConfig()
model_manager = ModelManager()
token_counter = TokenCounter()

In [11]:
class CustomJSONEncoder(json.JSONEncoder):
    """Custom JSON encoder to handle Pydantic models and other complex objects"""
    
    def default(self, obj):
        # Handle Pydantic models
        if hasattr(obj, 'model_dump'):
            return obj.model_dump()
        elif hasattr(obj, 'dict'):
            return obj.dict()
        # Handle datetime objects
        elif isinstance(obj, datetime):
            return obj.isoformat()
        # Handle Path objects
        elif isinstance(obj, Path):
            return str(obj)
        # Handle other non-serializable objects
        try:
            return super().default(obj)
        except TypeError:
            return str(obj)

def safe_json_dump(obj, file_path: Path, **kwargs):
    """Safely dump JSON with custom encoder"""
    try:
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(obj, f, cls=CustomJSONEncoder, ensure_ascii=False, **kwargs)
        return True
    except Exception as e:
        logger.error(f"Failed to write JSON to {file_path}: {e}")
        return False

In [12]:
class Event(BaseModel):
  source_text: str
  event: str
  event_who: str
  event_when: str
  event_what: str
  event_type: str

class EventList(BaseModel):
  events: list[Event]

In [13]:
models = ["gemma3:12b",
          "mistral:latest"
]
"""
models = ["gemma3:1b",
          "gemma3:12b",
          "chevalblanc/claude-3-haiku:latest",
          "incept5/llama3.1-claude:latest",
          "llama3.3:latest",
          "deepseek-r1:8b",
          "mistral:latest"
]
"""

event_definitions = """
You are an expert in legal text analysis. Here are the definitions of legal events:
- Event: Relates to the extent of text containing contextual event-related information. 
- Event_who: Corresponds to the subject of the event, which can either be a subject, but also an object (i.e., an application). 
    Examples: applicant, respondent, judge, witness
- Event_what: Corresponds to the main verb reflecting the baseline of all the paragraph. Additionally, we include thereto a complementing verb or object whenever the core verb is not self-explicit or requires an extension to attain a sufficient meaning.
    Examples: lodged an application, decided, ordered, dismissed
- Event_when: Refers to the date of the event, or to any temporal reference thereto.
- Event_circumstance: Meaning that the event correspond to the facts under judgment.
- Event_procedure: The events belongs to the procedural dimension of the case.

Events contain the annotations event_who, event_what and event_when. Events can be of type event_circumstance and event_procedure.
"""

instruction = "Analyze the provided text and extract the legal events. Provide the results in a structured format. Obviously, Event_who, Event_what and Event_when can only appear within an Event. If you find an event, also classify it into an event_circumstance or event_procedure. Do not invent additional information."


In [14]:
try:
    load_dotenv()

    user_email = os.getenv("USEREMAIL")  # Enter your email here
    password = os.getenv("PASSWORD")  # Enter your password here

    # Fetch Access Token

    # Define the URL for the authentication endpoint
    auth_url = "http://localhost:8080/api/v1/auths/signin"

    # Define the payload with user credentials
    auth_payload = json.dumps({"email": user_email, "password": "admin"})

    # Define the headers for the authentication request
    auth_headers = {"accept": "application/json", "content-type": "application/json"}

    # Make the POST request to fetch the access token
    auth_response = requests.post(auth_url, data=auth_payload, headers=auth_headers)

    # Extract the access token from the response
    access_token = auth_response.json().get("token")
except Exception as e:
    pass

In [15]:
def askChatbotImproved(model: str, role: str, instruction: str, content: str, 
                      max_retries: int = 3, retry_delay: float = 1.0) -> Optional[Dict[str, Any]]:
    """
    Improved chatbot function with better error handling and retries
    """
    chat_url = "http://localhost:11434/api/chat"
    
    # Check if content fits in model context
    if not model_manager.can_process_text(model, f"{role}\n{instruction}\n{content}", config.reserve_tokens):
        logger.warning(f"Text too long for model {model} context. Tokens: {token_counter.count_tokens(content)}")
        return None
    
    for attempt in range(max_retries):
        try:
            chat_headers = {
                "accept": "application/json",
                "content-type": "application/json",
                "Authorization": f"Bearer {access_token}",
            }
            
            chat_payload = {
                "stream": False,
                "model": model,
                "temperature": config.temperature,
                "messages": [
                    {"role": "system", "content": role},
                    {"role": "user", "content": f"{instruction}\n\n{content}"},
                ],
            }
            
            response = requests.post(chat_url, json=chat_payload, headers=chat_headers, timeout=120)
            response.raise_for_status()
            
            response_data = response.json()
            content = response_data.get("message", {}).get("content", "")
            
            if content:
                # Validate JSON structure
                try:
                    structured_response = EventList.model_validate_json(content)
                    logger.info(f"Successfully processed with {model} on attempt {attempt + 1}")
                    return {
                        "content": content, 
                        "structured": structured_response.model_dump() if hasattr(structured_response, 'model_dump') else structured_response.dict()
                    }
                except Exception as validation_error:
                    logger.warning(f"Validation error with {model}: {validation_error}")
                    return {"content": content, "structured": None}
            else:
                logger.warning(f"Empty response from {model}")
                
        except requests.exceptions.RequestException as e:
            logger.error(f"Request error with {model} (attempt {attempt + 1}): {e}")
        except Exception as e:
            logger.error(f"Unexpected error with {model} (attempt {attempt + 1}): {e}")
        
        if attempt < max_retries - 1:
            logger.info(f"Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
    
    logger.error(f"Failed to get response from {model} after {max_retries} attempts")
    return None

def askChatbotLocalImproved(model: str, role: str, instruction: str, content: str, 
                           max_retries: int = 3, retry_delay: float = 1.0) -> Optional[Dict[str, Any]]:
    """
    Improved local chatbot function with better error handling
    """
    # Check if content fits in model context
    if not model_manager.can_process_text(model, f"{role}\n{instruction}\n{content}", config.reserve_tokens):
        logger.warning(f"Text too long for model {model} context. Tokens: {token_counter.count_tokens(content)}")
        return None
    
    for attempt in range(max_retries):
        try:
            response = ollama.chat(
                model=model,
                options={'temperature': config.temperature},
                format=EventList.model_json_schema(),
                messages=[
                    {"role": "system", "content": role},
                    {"role": "user", "content": f"{instruction}\n\n{content}"},
                ]
            )
            
            content = response['message']['content']
            
            if content:
                try:
                    structured_response = EventList.model_validate_json(content)
                    logger.info(f"Successfully processed with {model} on attempt {attempt + 1}")
                    return {
                        "content": content, 
                        "structured": structured_response.model_dump() if hasattr(structured_response, 'model_dump') else structured_response.dict()
                    }
                except Exception as validation_error:
                    logger.warning(f"Validation error with {model}: {validation_error}")
                    return {"content": content, "structured": None}
            else:
                logger.warning(f"Empty response from {model}")
                
        except Exception as e:
            logger.error(f"Error with {model} (attempt {attempt + 1}): {e}")
        
        if attempt < max_retries - 1:
            logger.info(f"Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
    
    logger.error(f"Failed to get response from {model} after {max_retries} attempts")
    return None

In [16]:
def extract_document_sections(doc) -> Dict[str, str]:
    """
    Extract sections from a document with proper error handling
    """
    sections = {
        "procedure": "",
        "circumstances": "",
        "decision": ""
    }
    
    try:
        annotations = doc.annset("Section")
        if not annotations:
            logger.warning("No Section annotations found in document")
            return sections
        
        # Extract each section type
        section_types = [
            ("Procedure", "procedure"),
            ("Circumstances", "circumstances"),
            ("Decision", "decision")
        ]
        
        for gate_type, key in section_types:
            section_annotations = annotations.with_type(gate_type)
            if section_annotations:
                texts = []
                for ann in section_annotations:
                    text = doc.text[ann.start:ann.end]
                    if text.strip():
                        texts.append(text.strip())
                sections[key] = " ".join(texts)
            else:
                logger.warning(f"No {gate_type} annotations found")
        
        return sections
        
    except Exception as e:
        logger.error(f"Error extracting sections: {e}")
        return sections

def save_results_improved(doc_dict: Dict[str, Any], backup: bool = True) -> bool:
    """
    Improved save function with better error handling and backup
    """
    try:
        # Process events in annotations
        for ann in doc_dict.get("annotations", []):
            # Handle structured_events (Pydantic objects)
            if "structured_events" in ann and ann["structured_events"] is not None:
                try:
                    # Convert Pydantic model to dict
                    if hasattr(ann["structured_events"], 'model_dump'):
                        ann["structured_events"] = ann["structured_events"].model_dump()
                    elif hasattr(ann["structured_events"], 'dict'):
                        ann["structured_events"] = ann["structured_events"].dict()
                    else:
                        # If it's already a dict or other serializable type, keep as is
                        pass
                except Exception as e:
                    logger.warning(f"Failed to serialize structured_events: {e}")
                    ann["structured_events"] = None
            
            # Handle events string parsing
            if "events" in ann and isinstance(ann["events"], str):
                try:
                    parsed = json.loads(ann["events"])
                    if isinstance(parsed, dict) and "events" in parsed:
                        ann["events"] = parsed["events"]
                    else:
                        ann["events"] = parsed
                except json.JSONDecodeError as e:
                    logger.warning(f"Failed to parse events JSON: {e}")
                    ann["events"] = []
        
        # Generate clean document name
        doc_name = doc_dict.get("Document", "unknown")
        if isinstance(doc_name, str):
            doc_name = doc_name.replace("file:/C:/Users/mnavas/CASE%20OF%20", "")
            doc_name = doc_name.replace(".docx", "").replace("%20", " ")
            doc_name = doc_name.replace("/", "_").replace("\\", "_")
        
        # Add metadata
        doc_dict["metadata"] = {
            "processed_at": datetime.now().isoformat(),
            "total_sections": len([k for k in doc_dict.keys() if k in ["procedure", "circumstances", "decision"]]),
            "total_models": len(doc_dict.get("annotations", [])),
            "total_tokens": doc_dict.get("total_tokens", 0)
        }
        
        # Save to main output
        output_path = Path(config.output_dir) / f"{doc_name}.json"
        if not safe_json_dump(doc_dict, output_path, indent=2):
            return False
        
        # Create backup if requested
        if backup:
            backup_path = Path(config.backup_dir) / f"{doc_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
            safe_json_dump(doc_dict, backup_path, indent=2)
        
        logger.info(f"Successfully saved results for {doc_name}")
        return True
        
    except Exception as e:
        logger.error(f"Error saving results: {e}")
        return False

def process_document_with_models(doc, models: List[str], event_definitions: str, 
                                instruction: str) -> Dict[str, Any]:
    """
    Process a single document with multiple models
    """
    doc_name = doc.features.get("gate.SourceURL", "unknown")
    logger.info(f"Processing document: {doc_name}")
    
    # Extract sections
    sections = extract_document_sections(doc)
    combined_text = " ".join([text for text in sections.values() if text])
    
    if not combined_text.strip():
        logger.warning(f"No text extracted from document: {doc_name}")
        return None
    
    # Count tokens
    total_tokens = token_counter.count_tokens(combined_text)
    
    doc_dict = {
        "Document": doc_name,
        "sections": sections,
        "combined_text_length": len(combined_text),
        "total_tokens": total_tokens,
        "annotations": []
    }
    
    # Process with each model
    for model in models:
        logger.info(f"Processing with model: {model}")
        
        # Choose appropriate function based on configuration
        if config.via_web:
            response = askChatbotImproved(model, event_definitions, instruction, combined_text)
        else:
            response = askChatbotLocalImproved(model, event_definitions, instruction, combined_text)
        
        if response:
            annotation = {
                "model_name": model,
                "events": response["content"],
                "structured_events": response.get("structured"),
                "processed_at": datetime.now().isoformat(),
                "context_length": model_manager.get_context_length(model),
                "input_tokens": total_tokens
            }
            doc_dict["annotations"].append(annotation)
        else:
            logger.warning(f"Failed to get response from {model} for document {doc_name}")
    
    return doc_dict

In [ ]:
# Updated model configuration
# models = [
#     "gemma3:12b",
#     "mistral:latest"
# ]

# You can add more models as needed
models = [
    "gemma3:1b",
    "gemma3:4b",
    "gemma3:12b",
    "llama3.3:latest",
    "deepseek-r1:8b",
    "mistral:latest",
    "incept5/llama3.1-claude:latest", 
    "chevalblanc/claude-3-haiku:latest",
    "llama4:16x17b",
    "mixtral:8x7b"
]

def run_improved_pipeline(max_documents: int = 10, models: List[str] = None) -> Dict[str, Any]:
    """
    Run the improved information extraction pipeline
    """
    # if models is None:
    #     models = models
    
    logger.info(f"Starting IE pipeline with {len(models)} models and max {max_documents} documents")
    
    # Load corpus
    try:
        corpus = loadCorpus()
        logger.info(f"Loaded corpus with {len(corpus)} documents")
    except Exception as e:
        logger.error(f"Failed to load corpus: {e}")
        return {"error": str(e)}
    
    # Initialize results tracking
    results = {
        "processed_documents": 0,
        "failed_documents": 0,
        "total_annotations": 0,
        "start_time": datetime.now().isoformat(),
        "models_used": models,
        "documents": []
    }
    
    # Process documents
    for doc_idx, doc in enumerate(tqdm(corpus, desc="Processing documents")):
        if doc_idx >= max_documents:
            break
            
        try:
            doc_dict = process_document_with_models(doc, models, event_definitions, instruction)
            
            if doc_dict:
                # Save results
                if save_results_improved(doc_dict, backup=True):
                    results["processed_documents"] += 1
                    results["total_annotations"] += len(doc_dict.get("annotations", []))
                    results["documents"].append(doc_dict["Document"])
                else:
                    results["failed_documents"] += 1
            else:
                results["failed_documents"] += 1
                logger.warning(f"Failed to process document {doc_idx}")
                
        except Exception as e:
            logger.error(f"Error processing document {doc_idx}: {e}")
            results["failed_documents"] += 1
    
    # Finalize results
    results["end_time"] = datetime.now().isoformat()
    results["total_processing_time"] = str(datetime.fromisoformat(results["end_time"]) - datetime.fromisoformat(results["start_time"]))
    
    # Save pipeline results
    pipeline_results_path = Path(config.output_dir) / f"pipeline_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    safe_json_dump(results, pipeline_results_path, indent=2)
    
    logger.info(f"Pipeline completed. Processed: {results['processed_documents']}, Failed: {results['failed_documents']}")
    
    return results

In [ ]:
# Execute the improved pipeline
print("Running improved IE pipeline...")
print("=" * 50)

# Configure processing
config.max_documents = 1  # Start with a small number for testing
config.via_web = False    # Use local models
config.max_retries = 3
config.retry_delay = 2.0

# Run the pipeline
results = run_improved_pipeline(
    max_documents=config.max_documents,
    models=models
)

# Display results
print("\nPipeline Results:")
print("=" * 50)
print(f"Documents processed: {results['processed_documents']}")
print(f"Documents failed: {results['failed_documents']}")
print(f"Total annotations: {results['total_annotations']}")
print(f"Models used: {', '.join(results['models_used'])}")
print(f"Processing time: {results['total_processing_time']}")

if results['documents']:
    print(f"\nProcessed documents:")
    for doc in results['documents']:
        print(f"  - {doc}")

2025-07-11 18:15:36,147|INFO|__main__|Starting IE pipeline with 2 models and max 1 documents


Running improved IE pipeline...
Loaded input/updated/annotated\dev\CASE OF ALTAY v. TURKEY (No. 2).xml into corpus
Loaded input/updated/annotated\dev\CASE OF BELYAYEV AND OTHERS v. UKRAINE.xml into corpus
Loaded input/updated/annotated\dev\CASE OF BIGUN v. UKRAINE.xml into corpus
Loaded input/updated/annotated\test\CASE OF CABUCAK v. GERMANY.xml into corpus
Loaded input/updated/annotated\test\CASE OF CAN v. TURKEY.xml into corpus
Loaded input/updated/annotated\test\CASE OF CRISTIAN CATALIN UNGUREANU v. ROMANIA.xml into corpus
Loaded input/updated/annotated\train\CASE OF DOKTOROV v. BULGARIA.xml into corpus
Loaded input/updated/annotated\train\CASE OF EGILL EINARSSON v. ICELAND (No. 2).xml into corpus
Loaded input/updated/annotated\train\CASE OF HOINESS v. NORWAY.xml into corpus
Loaded input/updated/annotated\train\CASE OF KOSAITE - CYPIENE AND OTHERS v. LITHUANIA.xml into corpus
Loaded input/updated/annotated\train\CASE OF LOZOVYYE v. RUSSIA.xml into corpus
Loaded input/updated/annotat

2025-07-11 18:15:36,458|INFO|__main__|Loaded corpus with 30 documents


Loaded input/updated/annotated\train\CASE OF PAKHTUSOV v. RUSSIA.xml into corpus
Loaded input/updated/annotated\train\CASE OF PANYUSHKINY v. RUSSIA.xml into corpus
Loaded input/updated/annotated\train\CASE OF RESIN v. RUSSIA.xml into corpus
Loaded input/updated/annotated\train\CASE OF S.N. v. RUSSIA.xml into corpus
Loaded input/updated/annotated\train\CASE OF S.V. v. ITALY.xml into corpus
Loaded input/updated/annotated\train\CASE OF SHVIDKIYE v. RUSSIA.xml into corpus
Loaded input/updated/annotated\train\CASE OF SIDOROVA v. RUSSIA.xml into corpus
Loaded input/updated/annotated\train\CASE OF SOLCAN v. ROMANIA.xml into corpus
Loaded input/updated/annotated\train\CASE OF STANA v. ROMANIA.xml into corpus
Loaded input/updated/annotated\train\CASE OF VISY v. SLOVAKIA.xml into corpus
Loaded input/updated/annotated\train\CASE OF YAKUSHEV v. UKRAINE.xml into corpus
Loaded input/updated/annotated\train\CASE OF YERMAKOVICH v. RUSSIA.xml into corpus
Loaded input/updated/annotated\train\CASE OF YEV

Processing documents:   0%|          | 0/30 [00:00<?, ?it/s]2025-07-11 18:15:36,460|INFO|__main__|Processing document: file:/C:/Users/mnavas/CASE%20OF%20ALTAY%20v.%20TURKEY%20(No.%202).docx
2025-07-11 18:15:36,461|INFO|__main__|Processing with model: gemma3:12b
2025-07-11 18:15:36,460|INFO|__main__|Processing document: file:/C:/Users/mnavas/CASE%20OF%20ALTAY%20v.%20TURKEY%20(No.%202).docx
2025-07-11 18:15:36,461|INFO|__main__|Processing with model: gemma3:12b
2025-07-11 18:18:03,243|INFO|httpx|HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-07-11 18:18:03,244|INFO|__main__|Successfully processed with gemma3:12b on attempt 1
2025-07-11 18:18:03,244|INFO|__main__|Processing with model: mistral:latest
2025-07-11 18:18:03,243|INFO|httpx|HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-07-11 18:18:03,244|INFO|__main__|Successfully processed with gemma3:12b on attempt 1
2025-07-11 18:18:03,244|INFO|__main__|Processing with model: mistral:lat


Pipeline Results:
Documents processed: 1
Documents failed: 0
Total annotations: 2
Models used: gemma3:12b, mistral:latest
Processing time: 0:02:40.281864

Processed documents:
  - file:/C:/Users/mnavas/CASE%20OF%20ALTAY%20v.%20TURKEY%20(No.%202).docx


In [19]:
def analyze_results(output_dir: str = "output") -> Dict[str, Any]:
    """
    Analyze the results from the IE pipeline
    """
    output_path = Path(output_dir)
    json_files = list(output_path.glob("*.json"))
    
    if not json_files:
        logger.warning("No result files found")
        return {}
    
    analysis = {
        "total_files": len(json_files),
        "models_performance": {},
        "token_statistics": {},
        "event_statistics": {},
        "error_analysis": {}
    }
    
    all_docs = []
    
    for file_path in json_files:
        if file_path.name.startswith("pipeline_results_"):
            continue  # Skip pipeline summary files
            
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                doc_data = json.load(f)
                all_docs.append(doc_data)
        except Exception as e:
            logger.error(f"Error reading {file_path}: {e}")
    
    if not all_docs:
        return analysis
    
    # Analyze model performance
    for doc in all_docs:
        for annotation in doc.get("annotations", []):
            model_name = annotation.get("model_name", "unknown")
            if model_name not in analysis["models_performance"]:
                analysis["models_performance"][model_name] = {
                    "total_docs": 0,
                    "successful_responses": 0,
                    "failed_responses": 0,
                    "avg_tokens": 0,
                    "total_tokens": 0
                }
            
            analysis["models_performance"][model_name]["total_docs"] += 1
            
            if annotation.get("events"):
                analysis["models_performance"][model_name]["successful_responses"] += 1
            else:
                analysis["models_performance"][model_name]["failed_responses"] += 1
            
            tokens = annotation.get("input_tokens", 0)
            analysis["models_performance"][model_name]["total_tokens"] += tokens
    
    # Calculate averages
    for model_stats in analysis["models_performance"].values():
        if model_stats["total_docs"] > 0:
            model_stats["avg_tokens"] = model_stats["total_tokens"] / model_stats["total_docs"]
            model_stats["success_rate"] = (model_stats["successful_responses"] / model_stats["total_docs"]) * 100
    
    # Token statistics
    token_counts = [doc.get("total_tokens", 0) for doc in all_docs]
    if token_counts:
        analysis["token_statistics"] = {
            "mean": sum(token_counts) / len(token_counts),
            "min": min(token_counts),
            "max": max(token_counts),
            "total": sum(token_counts)
        }
    
    return analysis

def display_analysis(analysis: Dict[str, Any]):
    """
    Display analysis results in a formatted way
    """
    print("ANALYSIS RESULTS")
    print("=" * 60)
    
    print(f"Total files analyzed: {analysis.get('total_files', 0)}")
    
    print("\nModel Performance:")
    print("-" * 40)
    for model, stats in analysis.get("models_performance", {}).items():
        print(f"{model}:")
        print(f"  Documents processed: {stats['total_docs']}")
        print(f"  Success rate: {stats.get('success_rate', 0):.1f}%")
        print(f"  Average tokens: {stats['avg_tokens']:.0f}")
        print()
    
    token_stats = analysis.get("token_statistics", {})
    if token_stats:
        print("Token Statistics:")
        print("-" * 40)
        print(f"  Mean tokens per document: {token_stats['mean']:.0f}")
        print(f"  Min tokens: {token_stats['min']}")
        print(f"  Max tokens: {token_stats['max']}")
        print(f"  Total tokens processed: {token_stats['total']:,}")

# Run analysis
print("\nAnalyzing results...")
analysis = analyze_results()
display_analysis(analysis)


Analyzing results...
ANALYSIS RESULTS
Total files analyzed: 2

Model Performance:
----------------------------------------
gemma3:12b:
  Documents processed: 1
  Success rate: 100.0%
  Average tokens: 2210

mistral:latest:
  Documents processed: 1
  Success rate: 100.0%
  Average tokens: 2210

Token Statistics:
----------------------------------------
  Mean tokens per document: 2210
  Min tokens: 2210
  Max tokens: 2210
  Total tokens processed: 2,210
